In [3]:
import netCDF4 as nc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os
import xarray as xr
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import pyproj
from scipy.interpolate import griddata
import cftime
from sklearn.linear_model import LinearRegression

In [6]:
# ERA5
home_path = os.path.expanduser("~")

path = '/DataFiles'
path2 = '/Library/Mobile Documents/com~apple~CloudDocs/Documents/Documents - MacBook Air/Python/Ice Cores/data/model/ccsm4_last_millenium'

allEra = nc.Dataset(home_path + path2 + '/era5total.nc')

In [ ]:
home_path = os.path.expanduser("~")

path = '/DataFiles'
era = xr.open_dataset(home_path + path + "/ae3baa6a74f0aa315dc3de6f83298f0e.nc")

path2 = '/Library/Mobile Documents/com~apple~CloudDocs/Documents/Documents - MacBook Air/Python/Ice Cores/data/model/ccsm4_last_millenium'


In [7]:
print(allEra.variables['tp'])

<class 'netCDF4.Variable'>
int16 tp(time, expver, latitude, longitude)
    scale_factor: 4.09993986774206e-07
    add_offset: 0.013434486031357766
    _FillValue: -32767
    missing_value: -32767
    units: m
    long_name: Total precipitation
unlimited dimensions: 
current shape = (547, 2, 301, 3600)
filling on


In [33]:
print(allEra.variables['time'][0])

692496


In [18]:
#ERA5 Data processing
tasBase = np.array(allEra.variables['t2m'][:276, 0])
prBase = np.array(allEra.variables['tp'][:276, 0])

eraTemps = np.zeros([22, 301, 3600])
eraAccums = np.zeros([22, 301, 3600])

for i in range(22):
    eraTemps[i] = np.mean(tasBase[(i*12):(i*12+12)], axis=0)
    eraAccums[i] = np.mean(prBase[(i*12):(i*12+12)], axis=0) * 365000

eraAccums[eraAccums < 0] = np.nan

# Regridding to the resolution of CCSM4
rolledTemps = np.roll(eraTemps, 1800, axis=2)
rolledAccums = np.roll(eraAccums, 1800, axis=2)

tempRegrid = np.zeros([22, 32, 3600])
secondRegrid = np.zeros([22, 32, 7200])
latInd = np.array(pd.read_csv(home_path + path2 + '/latInd.csv'))

firstTemps = np.zeros([22, 32, 3600])
firstAccums = np.zeros([22, 32, 3600])

secondTemps = np.zeros([22, 32, 7200])
secondAccums = np.zeros([22, 32, 7200])

i = 0
for item in latInd:
    firstTemps[:, i, :] = rolledTemps[:, item[0]]
    firstAccums[:, i, :] = rolledAccums[:, item[0]]
    i = i + 1

for j in range(22):
    for i in range(32):
        secondTemps[j, i, :7198] = np.dstack((firstTemps[j, i, :-1], firstTemps[j, i, :-1] + np.diff(firstTemps[j, i]) / 2.0)).ravel()
        secondAccums[j, i, :7198] = np.dstack((firstAccums[j, i, :-1], firstAccums[j, i, :-1] + np.diff(firstAccums[j, i]) / 2.0)).ravel()

finalTemps = secondTemps[:, :, ::25]
finalAccums = secondAccums[:, :, ::25]



In [27]:
basins = np.loadtxt(home_path + path2 + '/ant_full_drainagesystem_polygons.txt')

        
basinAvgs = np.zeros(27)
b = 1
tempSum = 0
count = 0


basNo = 27
yearSums = np.zeros(22)
yearCount = np.zeros(22)
rSums = np.zeros(28)
ceSums = np.zeros(28)
cellCount = 0.0

areas = [0, 501448, 830000, 1565727, 247879, 190978, 610797, 493593, 159742, 146029, 919041, 257286, 722224, 1109771, 710953, 125183, 265243, 1852382, 269776, 382018, 201853, 217404, 214497, 91266, 160848, 34621, 43085, 54146]
basinMaster = np.zeros([27, 22])

for k in range(28):
    if k != 0:
        for i in basins:
            if i[2] == k:
                lat = i[0]
                lon = i[1]

                if lon < 0:
                    lon = lon + 360

                latInd = round(lat / 2) + 45
                lonInd = round(lon / 2)

                if lonInd == 180:
                    lonInd = 0


                if finalTemps[0, int(latInd), int(lonInd)] > 0.0:
                    for j in range(22):
                        yearCount[j] = yearCount[j] + 1
                        #yearSums[j] = yearSums[j] + finalAccums[j, int(latInd), int(lonInd)]
                        yearSums[j] = yearSums[j] + finalTemps[j, int(latInd), int(lonInd)]

        timeSeries = np.divide(yearSums, yearCount)
        for i in range(22):
            basinMaster[k-1, i] = 0.9167*(timeSeries[i]*areas[k-1])/1000000
        print(k)
        yearSums = np.zeros(22)
        yearCount = np.zeros(22)
        b = 1
        tempSum = 0
        count = 0
        basinAvgs = np.zeros(27)

AP = np.sum(basinMaster[23:], axis=0)
WAIS = np.sum(basinMaster[17:23], axis=0) + basinMaster[0]
EAIS = np.sum(basinMaster[1:17], axis=0) 
AIS = AP + WAIS + EAIS

for i in range(22):
    print(AIS[i])

#np.savetxt("ERA5AccumMetric_AIS.csv", AIS)
#np.savetxt("ERA5AccumMetric_WAIS.csv", WAIS)
#np.savetxt("ERA5AccumMetric_EAIS.csv", EAIS)
#np.savetxt("ERA5AccumMetric_AP.csv", AP)


avgAIS = np.zeros([1, 22])
avgWAIS = np.zeros([1, 22])
avgEAIS = np.zeros([1, 22])
avgAP = np.zeros([1, 22])
print("________________________")
for i in range(27):
    avgAIS[0, :] = avgAIS[0, :] + areas[i] * basinMaster[i]
    if i == 0 or (i > 16 and i < 23):
        avgWAIS[0, :] = avgWAIS[0, :] + areas[i] * basinMaster[i]
    elif i > 0 and i < 17:
        avgEAIS[0, :] = avgEAIS[0, :] + areas[i] * basinMaster[i]
    else:
        avgAP[0, :] = avgAP[0, :] + areas[i] * basinMaster[i]
    
avgAIS = avgAIS / np.sum(areas, axis=0)
avgWAIS = avgWAIS / (np.sum(areas[17:23], axis=0) + areas[0])
avgEAIS = avgEAIS / np.sum(areas[1:17], axis=0)
avgAP_2 = avgAP / np.sum(areas[23:], axis=0)
    
#AP = np.sum(basinMaster[23:], axis=0)
#WAIS = np.sum(basinMaster[17:23], axis=0) + basinMaster[0]
#EAIS = np.sum(basinMaster[1:17], axis=0) 
#AIS = AP + WAIS + EAIS

print(avgAP)

#np.savetxt("ERA5TempMetric_AIS.csv", avgAIS)
#np.savetxt("ERA5TempMetric_WAIS.csv", avgWAIS)
#np.savetxt("ERA5TempMetric_EAIS.csv", avgEAIS)
np.savetxt("ERA5TempMetric_AP.csv", avgAP)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
2603.173056024232
2624.6570551765135
2616.313504506702
2609.929731114429
2611.151460919309
2616.837556289084
2606.880586807766
2611.765391745781
2603.106569727941
2619.67600449955
2609.751987208352
2612.8406780776218
2619.4327649930638
2615.615414577439
2605.8079452077645
2605.3998219600594
2607.493798877817
2616.3878056792137
2605.866565669551
2606.0562046808795
2611.622332091737
2611.7397269651833
________________________
[[8355904.51623303 8492151.2874007  8382689.13957127 8341173.82738257
  8384880.94218003 8452985.19651063 8338480.75193327 8357027.39280799
  8385970.29343574 8448084.67866529 8355333.3803082  8341815.52743161
  8425369.9804829  8418633.38228931 8408661.72196856 8363153.24430406
  8389626.47780371 8401014.43486023 8393370.15752294 8387439.9011912
  8358116.81268461 8411853.76437056]]


In [26]:
print(np.sum(areas[23:], axis=0))

383966


In [28]:
print(avgAP / 383966)

[[21.76209486 22.11693558 21.83185266 21.7237303  21.83756099 22.01493152
  21.71671646 21.76501928 21.84039809 22.00216863 21.7606074  21.72540154
  21.94301053 21.92546575 21.89949559 21.78097343 21.84992025 21.87957901
  21.85967028 21.84422553 21.76785656 21.90780893]]
